### Método build_features('SamplesPath.txt', arraywords(Y)) 

Parametros: un array con las palabra y el path del archivo que contiene las rutas de los audios analizar

Retorna: Matriz con #filas: cantidad de audios, #columna: cantidad de bloques por audio +2 columnas adicionales donde en la primera es la clase a la que pertenece y la segunda es el id del hablante


In [94]:
! pip install ipynb

In [58]:
import IPython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
from get_features_audio import GETFEATURES
%matplotlib inline

### CREAR DATASET PARA ENTRENAR Y VALIDAR

In [128]:
def build_features(PATHSAMPLES, SOUNDS):
    GF = GETFEATURES()
    ROOTPATH= 'data/'
    data = pd.read_csv(PATHSAMPLES, sep=" ", header=None)
    data = data.values #Convertir en numpy array
    labels = []
    speakers  = []
    paths = []
    DIVISIONROW, DIVISIONCOL = 9,9
    for path in data:
        pathSplit =path[0].split('/')
        speakerName = pathSplit[1].split('_')[0]
        word = pathSplit[0]
        isWord = np.where(SOUNDS == word)
        if len(isWord[0]) != 0:
            label = SOUNDS[isWord[0][0]]
            labels.append(label)
            speakers.append(speakerName)
            paths.append(ROOTPATH+path[0])
    
    #bloques y ademas una columna para la etiqueta o sonido que represente y otra para el id del hablante
    #NUMERO DE MUESTRAS POR CLASE
    NSAMPLES = 1300
    first = labels.index(SOUNDS[0])
    last = first +NSAMPLES
    matrixMajor = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
    for i in range(1,np.size(SOUNDS,0)):
        first = labels.index(SOUNDS[i])
        last = first +NSAMPLES
        clasei = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
        matrixMajor = np.append(matrixMajor,clasei,axis=0)
        # Contienes todos los paths , palabra y idSpeaker de 1300 por clase
    
    
    rows = matrixMajor.shape[0] # numero de filas es la cantidad de audios 
    colums = DIVISIONROW*DIVISIONCOL*2+2 # cantidad de bloques por 2 ya que esta la media y std de todos los 
    
    #Recorre la matriz, importar el metodo get_features_audio del archivo get_features_audio, extraer las 5200 features, codificar la columna de label y guardad en un archivo
    matrixFeatures = np.ones((rows,colums))
    cont = 0
    #Encode etiqueta del audio y el speaker
    leLabel = preprocessing.LabelEncoder()
    leLabel.fit(matrixMajor[:,1])
    leSpeaker = preprocessing.LabelEncoder()
    leSpeaker.fit(matrixMajor[:,2])
    #----------------------
    
    for infoSound in matrixMajor:
        feat = GF.get_features_audios(infoSound[0], DIVISIONROW, DIVISIONCOL)
        label = infoSound[1]
        speaker = infoSound[2]
        labelN = leLabel.transform([label])
        labelS = leSpeaker.transform([speaker])
        matrixFeatures[cont][0:colums-2]=feat
        matrixFeatures[cont][-2]=labelS[0]
        matrixFeatures[cont][-1]=labelN[0]
        cont=cont+1
    return matrixFeatures   

In [131]:
matrixFeatures.shape

(5200, 164)

### Guardar dataset en un archivo csv

In [129]:
#PATHSAMPLES = './training_list.txt'
#SOUNDS =np.array(['backward','bed','cat','eight'])
#matrixFeatures = build_features(PATHSAMPLES,SOUNDS)
#np.savetxt('data/audiosData.csv',matrixFeatures,delimiter=",")